In [1]:
import torch
from torch import Tensor
from torch.nn.functional import softmax
from captum.attr import LayerIntegratedGradients
from captum.attr import visualization as viz
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BatchEncoding, PreTrainedTokenizer

In [2]:
# device, model, tokenizerを準備
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_name = "abhishek/autonlp-japanese-sentiment-59363"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
# modelを使って予測する関数を定義
def predict(
    input_ids: Tensor,
    token_type_ids: Tensor,
    attention_mask: Tensor,
) -> Tensor:
    return model(
        input_ids=input_ids,
        token_type_ids=token_type_ids,
        attention_mask=attention_mask,
    ).logits

In [4]:
# predict関数への入力を準備する関数を定義
def prepare_input(tokenizer: PreTrainedTokenizer, text: str, device: torch.device) -> BatchEncoding:
    return tokenizer(
        text, truncation=True, max_length=512, return_tensors="pt"
    ).to(device)

In [5]:
# 各単語埋め込みの各次元の予測への寄与度から、各単語埋め込みの予測への寄与度を計算する関数を定義
# [入力系列長 x 単語埋め込みの次元]の行列から[入力系列長]の行列に変換
# 単語埋め込みの寄与度には、単語埋め込みの各次元の予測への寄与度の合計を利用
def summarize_attributions(attributions: Tensor) -> Tensor:
    attributions = attributions.sum(dim=-1).flatten()
    attributions = attributions / torch.norm(attributions)
    return attributions

In [6]:
# 予測を行う関数と、Integrated Gradientsを計算する層を渡す
lig = LayerIntegratedGradients(predict, model.bert.embeddings)

In [7]:
# 分析対象のインスタンスを準備
input_text = "この服は着心地が良く、購入して大正解でした！ただ、梱包が悪かった部分は残念でした..."
gold_label = 1

In [8]:
# 入力テキストをエンコード
ids = prepare_input(tokenizer, input_text, device)

# ベースラインとなる入力を作成
# エンコードした入力テキストのうち、[CLS]と[SEP]以外のトークンを[PAD]に置き換える
baselines = ids.input_ids.clone()
baselines[(baselines != tokenizer.cls_token_id) * (baselines != tokenizer.sep_token_id)] = tokenizer.pad_token_id

# Integrated Gradientsを計算
attributions, delta = lig.attribute(
    # 入力のうち、寄与度を計算したい入力を指定
    inputs=ids.input_ids,

    baselines=baselines,
    
    # 入力のうち、寄与度を計算しない入力を指定
    additional_forward_args=(ids.token_type_ids, ids.attention_mask),
    return_convergence_delta=True,
    
    # 正解ラベルを指定 (回帰タスクの場合は不要)
    target=gold_label
)
attributions_sum = summarize_attributions(attributions)

In [9]:
score = predict(**ids)
pred_prob = softmax(score, dim=1).flatten().max().item()
pred_label = score.argmax().item()

tokens = tokenizer.convert_ids_to_tokens(ids.input_ids[0])

# 可視化のための情報を集約
result_vis = viz.VisualizationDataRecord(
    attributions_sum,
    pred_prob,
    pred_label,
    gold_label,
    gold_label,
    attributions_sum.sum(),
    tokens,
    delta
)

In [10]:
# 可視化
viz.visualize_text([result_vis])

In [11]:
# Integrated Gradientsによる分析は、ラップするクラスを作成すると便利
class IntegratedGradientsForBert:
    def __init__(self, model_name: str, gpu: int):
        self.model_name = model_name
        self.gpu = gpu
        
        self.device = torch.device(f"cuda:{self.gpu}" if torch.cuda.is_available() else "cpu")

        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name).to(self.device)
        self.tokenizer: PreTrainedTokenizer = AutoTokenizer.from_pretrained(self.model_name)

        self.lig = LayerIntegratedGradients(
            self.predict_with_model, self.model.bert.embeddings
        )

    def predict_with_model(self, input_ids: Tensor, token_type_ids: Tensor, attention_mask: Tensor) -> Tensor:
        return self.model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask
        ).logits

    def prepare_inputs(self, text: str) -> BatchEncoding:
        return self.tokenizer(
            text, truncation=True, max_length=512, return_tensors="pt"
        ).to(self.device)

    @staticmethod
    def summarize_attributions(attributions: Tensor) -> Tensor:
        attributions = attributions.sum(dim=-1).squeeze(0)
        attributions = attributions / torch.norm(attributions)
        return attributions

    def calculate_summarized_attributions(self, ids: BatchEncoding, gold_class: int) -> tuple[Tensor, Tensor]:
        # baseline: [CLS][PAD][PAD]...[SEP][PAD][PAD]...[SEP]
        baselines = ids.input_ids.clone()
        baselines[(baselines != self.tokenizer.cls_token_id) * (baselines != self.tokenizer.sep_token_id)] = self.tokenizer.pad_token_id

        attributions, delta = self.lig.attribute(
            inputs=ids.input_ids,
            baselines=baselines,
            additional_forward_args=(ids.token_type_ids, ids.attention_mask),
            return_convergence_delta=True,
            target=gold_class
        )

        summarized_attributions = self.summarize_attributions(attributions)

        return summarized_attributions, delta

    def generate_attributions_for_visualize(self, text: str, gold_class: int) -> tuple[Tensor, Tensor, float, list[str]]:
        ids = self.prepare_inputs(text)

        summarized_attributions, delta = self.calculate_summarized_attributions(ids, gold_class)
        prediction = self.predict_with_model(**ids)
        tokens = self.tokenizer.convert_ids_to_tokens(ids.input_ids[0])

        return summarized_attributions, delta, prediction, tokens

    def visualize_attributions(self, text: str, gold_class: int):
        summarized_attributions, delta, prediction, tokens = self.generate_attributions_for_visualize(text, gold_class)

        pred_prob = softmax(prediction, dim=1).flatten().max().item()
        pred_class = prediction.argmax().item()

        attributions_for_visualization = viz.VisualizationDataRecord(
            word_attributions=summarized_attributions,
            pred_prob=pred_prob,
            pred_class=pred_class,
            true_class=gold_class,
            attr_class=gold_class,
            attr_score=summarized_attributions.sum(),
            raw_input_ids=tokens,
            convergence_score=delta
        )

        viz.visualize_text([attributions_for_visualization])

In [12]:
model_name = "abhishek/autonlp-japanese-sentiment-59363"
gpu = 0

igm = IntegratedGradientsForBert(model_name, gpu)

In [13]:
input_text = "この服は着心地が良く、購入して大正解でした！ただ、梱包が悪かった部分は残念でした..."
gold_class = 1

igm.visualize_attributions(input_text, gold_class)